# Analyze Lambda Logs in CloudWatch

Get Data from CloudWatch

In [ ]:
import boto3
import pandas as pd
boto3.setup_default_session(profile_name='dev')
client = boto3.client('logs', region_name='us-east-1')

In [ ]:
groups=client.describe_log_groups()['logGroups']
report_events=[]
kmeans_events=[]
for i in groups:
    if i['logGroupName'].startswith("/aws/lambda"):
        response= client.filter_log_events(logGroupName=i['logGroupName'], filterPattern="REPORT")
        report_events.extend(response["events"])
        response= client.filter_log_events(logGroupName=i['logGroupName'], filterPattern="LambdaKMeans")
        kmeans_events.extend(response["events"])
        

In [ ]:
report_data=[i['message'].split("\t") for i in report_events]

In [ ]:
lines=[]
header=[]
for record in report_data:
    line = []
    header=[]
    for column in record:
        try:
            line.append(column.split(":")[1].strip().rstrip('MB').rstrip("ms").strip())
            header.append(column.split(":")[0].lstrip("REPORT").strip())
        except:
            pass
    lines.append(line)
        

In [ ]:
accounting=pd.DataFrame(lines, columns=header)
accounting

In [ ]:
kmeans_data=[i['message'].strip().split(",") for i in kmeans_events]
runtime = pd.DataFrame(kmeans_data, columns=["Measurement", "RequestID", "Log Group", "Log Stream", "Cores","Number Points", "Number Dimensions", "Processing Time", "Latency"])
runtime

In [ ]:
all_df = runtime.join(accounting)

In [ ]:
all_df

# Clean S3 and CloudWatch

In [ ]:
response=client.describe_log_groups()
for i in response['logGroups']:
    print("Delete %s"%str(i["logGroupName"]))
    client.delete_log_group(logGroupName=i["logGroupName"])

In [ ]:
import os
output=!aws s3 ls --profile dev | awk -F" " '{print $3}'
for i in output:
    print(i)
    os.system("aws s3 rm s3://%s --recursive --profile dev"%i)
    os.system("aws s3 rb s3://%s --profile dev"%i)